In [20]:
from __future__ import absolute_import, division, print_function, unicode_literals
# import libraries
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential 
from tensorflow.keras import layers
from tensorflow.keras import optimizers
# Helper libraries

print(tf.__version__)

2.4.1


In [26]:
path = os.getcwd() + '/final_data/vgsalesCleanedPublishers.csv'
data = pd.read_csv(path, header=0)

del data['Rank']
del data['Name']
#del data['Publisher']
del data['NA_Sales']
del data['EU_Sales']
del data['JP_Sales']
del data['Other_Sales']

target = data['Global_Sales']
del data['Global_Sales']

for t in range(len(target)):
    if target[t]> 10:
        target.at[t] = 9
    elif target[t]> 5:
       target.at[t]= 8
    elif target[t]>3:
       target.at[t]= 7
    elif target[t]>1:
       target.at[t]= 6
    elif target[t]>0.5:
       target.at[t]= 5
    elif target[t]>0.3:
       target.at[t]= 4
    elif target[t]>0.1:
       target.at[t]= 3
    elif target[t]>0.05:
       target.at[t]= 2
    elif target[t]>0.03:
       target.at[t]= 1
    elif target[t]>=0.01:
       target.at[t]= 0
print(target)

#new_target = np.zeros((target.shape[0], 10))
#for i in range(target.shape[0]):
#    new_target[i, int(target.at[i])] = 1

platforms = {}
for p in data['Platform']:
    if p in platforms.keys():
        pass
    else:
        platforms[p] = len(platforms)
print(platforms)

genres = {}
for g in data['Genre']:
    if g in genres.keys():
        pass
    else:
        genres[g] = len(genres)
print(genres)

publishers = {}
for pu in data['Publisher']:
    if pu in publishers.keys():
        pass
    else:
        publishers[pu] = len(publishers)
print(publishers)

for i in range(data.shape[0]):
    data.at[i,'Platform'] = platforms[data['Platform'][i]]
    data.at[i, 'Genre'] = genres[data['Genre'][i]]
    data.at[i,'Publisher'] = publishers[data['Publisher'][i]]
print(data)


0        9.0
1        9.0
2        9.0
3        9.0
4        9.0
        ... 
16431    0.0
16432    0.0
16433    0.0
16434    0.0
16435    0.0
Name: Global_Sales, Length: 16436, dtype: float64
{'Wii': 0, 'NES': 1, 'GB': 2, 'DS': 3, 'X360': 4, 'PS3': 5, 'PS2': 6, 'SNES': 7, 'GBA': 8, '3DS': 9, 'PS4': 10, 'N64': 11, 'PS': 12, 'XB': 13, 'PC': 14, '2600': 15, 'PSP': 16, 'XOne': 17, 'GC': 18, 'WiiU': 19, 'GEN': 20, 'DC': 21, 'PSV': 22, 'SAT': 23, 'SCD': 24, 'WS': 25, 'NG': 26, 'TG16': 27, '3DO': 28, 'GG': 29, 'PCFX': 30}
{'Sports': 0, 'Platform': 1, 'Racing': 2, 'Role-Playing': 3, 'Puzzle': 4, 'Misc': 5, 'Shooter': 6, 'Simulation': 7, 'Action': 8, 'Fighting': 9, 'Adventure': 10, 'Strategy': 11}
{'Nintendo': 0, 'Microsoft Game Studios': 1, 'Take-Two Interactive': 2, 'Sony Computer Entertainment': 3, 'Activision': 4, 'Ubisoft': 5, 'Bethesda Softworks': 6, 'Electronic Arts': 7, 'Sega': 8, 'SquareSoft': 9, 'Atari': 10, '505 Games': 11, 'Capcom': 12, 'GT Interactive': 13, 'Konami Digital Enterta

      Platform  Year Genre Publisher
0            0  2006     0         0
1            1  1985     1         0
2            0  2008     2         0
3            0  2009     0         0
4            2  1996     3         0
...        ...   ...   ...       ...
16431        8  2002     1       187
16432       18  2003     6        51
16433        6  2008     2         4
16434        3  2010     4       507
16435        8  2003     1       170

[16436 rows x 4 columns]


In [35]:
x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=2700, random_state=0)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=2700, random_state=0)
print(x_train.shape)

x_train = np.asarray(x_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)

x_test = np.asarray(x_test).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)

x_valid = np.asarray(x_valid).astype(np.float32)
y_valid = np.asarray(y_valid).astype(np.float32)

x_train = np.expand_dims(x_train, axis=1)

x_test = np.expand_dims(x_test, axis=1)

x_valid = np.expand_dims(x_valid, axis=1)

print(x_train[0])

(11036, 4)
[[   6. 2008.    3.   56.]]


In [75]:
first_activations = ['gelu', 'linear', 'exponential', 'relu', 'sigmoid']
second_activations = ['gelu', 'linear', 'exponential', 'relu', 'sigmoid']
third_activations = ['gelu', 'linear', 'exponential', 'relu', 'sigmoid']
losses = ['categorical_crossentropy', 'binary_crossentropy', 'mse']
batch_sizes = [16, 32, 64, 128]

accuracies = {}

for fa in first_activations:
    print(fa, '\n\n')
    accuracies[fa] = {}
    for sa in second_activations:
        print(sa)
        accuracies[fa][sa] = {}
        for ta in third_activations:
            accuracies[fa][sa][ta] = {}
            for los in losses:
                accuracies[fa][sa][ta][los] = {}
                for batch in batch_sizes:
                    model = keras.Sequential()
                    model.add(keras.layers.LSTM(100, activation=fa, return_sequences = True))
                    model.add(keras.layers.Dropout(.2))
                    model.add(keras.layers.LSTM(100, activation=sa, return_sequences = True))
                    model.add(keras.layers.Dropout(.2))
                    model.add(keras.layers.LSTM(50, activation=ta))
                    model.add(keras.layers.Dropout(.2))
                    model.add(keras.layers.Dense(1, activation='softmax'))
                    model.compile(optimizer=keras.optimizers.Adam(learning_rate = .001, decay=.00001), metrics=['accuracy'], loss=los)
    
                    model.fit(x_train, y_train, batch_size=batch, epochs=10, validation_data=(x_valid, y_valid))

                    accuracies[fa][sa][ta][los][batch] = model.evaluate(x_test, y_test, batch_size=batch)

gelu 


gelu
Epoch 1/10
690/690 [==============================] - 10s 5ms/step - loss: 0.0000e+00 - accuracy: 0.0794 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 2/10
690/690 [==============================] - 3s 4ms/step - loss: 0.0000e+00 - accuracy: 0.0741 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 3/10
690/690 [==============================] - 3s 5ms/step - loss: 0.0000e+00 - accuracy: 0.0716 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 4/10
690/690 [==============================] - 3s 5ms/step - loss: 0.0000e+00 - accuracy: 0.0736 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 5/10
690/690 [==============================] - 3s 5ms/step - loss: 0.0000e+00 - accuracy: 0.0781 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 6/10
690/690 [==============================] - 3s 5ms/step - loss: 0.0000e+00 - accuracy: 0.0771 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 7/10
690/690 [==============================] - 3s 5ms/step - loss: 0.0000e+00 -

345/345 [==============================] - 9s 7ms/step - loss: -876197.1930 - accuracy: 0.0781 - val_loss: -27969460.0000 - val_accuracy: 0.0789
Epoch 2/10
345/345 [==============================] - 2s 5ms/step - loss: -105915634.2775 - accuracy: 0.0717 - val_loss: -659201024.0000 - val_accuracy: 0.0789
Epoch 3/10
345/345 [==============================] - 2s 5ms/step - loss: -1149781300.1618 - accuracy: 0.0756 - val_loss: -3395508480.0000 - val_accuracy: 0.0789
Epoch 4/10
345/345 [==============================] - 2s 5ms/step - loss: -4821546945.8497 - accuracy: 0.0776 - val_loss: -10293879808.0000 - val_accuracy: 0.0789
Epoch 5/10
345/345 [==============================] - 2s 5ms/step - loss: -13262287650.0347 - accuracy: 0.0733 - val_loss: -23286804480.0000 - val_accuracy: 0.0789
Epoch 6/10
345/345 [==============================] - 2s 5ms/step - loss: -28704892229.5491 - accuracy: 0.0729 - val_loss: -44260810752.0000 - val_accuracy: 0.0789
Epoch 7/10
345/345 [======================

173/173 [==============================] - 8s 10ms/step - loss: 7.9906 - accuracy: 0.0767 - val_loss: 7.6648 - val_accuracy: 0.0789
Epoch 2/10
173/173 [==============================] - 1s 7ms/step - loss: 7.9935 - accuracy: 0.0744 - val_loss: 7.6648 - val_accuracy: 0.0789
Epoch 3/10
173/173 [==============================] - 1s 7ms/step - loss: 7.8680 - accuracy: 0.0761 - val_loss: 7.6648 - val_accuracy: 0.0789
Epoch 4/10
173/173 [==============================] - 1s 7ms/step - loss: 7.9393 - accuracy: 0.0777 - val_loss: 7.6648 - val_accuracy: 0.0789
Epoch 5/10
173/173 [==============================] - 1s 8ms/step - loss: 8.0473 - accuracy: 0.0729 - val_loss: 7.6648 - val_accuracy: 0.0789
Epoch 6/10
173/173 [==============================] - 1s 7ms/step - loss: 7.9782 - accuracy: 0.0792 - val_loss: 7.6648 - val_accuracy: 0.0789
Epoch 7/10
173/173 [==============================] - 1s 7ms/step - loss: 7.9673 - accuracy: 0.0766 - val_loss: 7.6648 - val_accuracy: 0.0789
Epoch 8/10
173/1

Epoch 4/10
87/87 [==============================] - 1s 9ms/step - loss: 0.0000e+00 - accuracy: 0.0746 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 5/10
87/87 [==============================] - 1s 9ms/step - loss: 0.0000e+00 - accuracy: 0.0799 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 6/10
87/87 [==============================] - 1s 9ms/step - loss: 0.0000e+00 - accuracy: 0.0747 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 7/10
87/87 [==============================] - 1s 10ms/step - loss: 0.0000e+00 - accuracy: 0.0770 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 8/10
87/87 [==============================] - 1s 9ms/step - loss: 0.0000e+00 - accuracy: 0.0791 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 9/10
87/87 [==============================] - 1s 9ms/step - loss: 0.0000e+00 - accuracy: 0.0809 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 10/10
22/22 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0800
Epoch 1/

690/690 [==============================] - 3s 4ms/step - loss: 7.9586 - accuracy: 0.0779 - val_loss: 7.6648 - val_accuracy: 0.0789
Epoch 3/10
690/690 [==============================] - 3s 4ms/step - loss: 7.8799 - accuracy: 0.0804 - val_loss: 7.6648 - val_accuracy: 0.0789
Epoch 4/10
690/690 [==============================] - 3s 4ms/step - loss: 7.8221 - accuracy: 0.0756 - val_loss: 7.6648 - val_accuracy: 0.0789
Epoch 5/10
690/690 [==============================] - 3s 4ms/step - loss: 7.9230 - accuracy: 0.0760 - val_loss: 7.6648 - val_accuracy: 0.0789
Epoch 6/10
690/690 [==============================] - 3s 4ms/step - loss: 8.0812 - accuracy: 0.0779 - val_loss: 7.6648 - val_accuracy: 0.0789
Epoch 7/10
690/690 [==============================] - 3s 4ms/step - loss: 7.9332 - accuracy: 0.0764 - val_loss: 7.6648 - val_accuracy: 0.0789
Epoch 8/10
690/690 [==============================] - 3s 4ms/step - loss: 8.1095 - accuracy: 0.0716 - val_loss: 7.6648 - val_accuracy: 0.0789
Epoch 9/10
690/69

...

Epoch 3/10
173/173 [==============================] - 1s 7ms/step - loss: 0.0000e+00 - accuracy: 0.0766 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 4/10
173/173 [==============================] - 1s 7ms/step - loss: 0.0000e+00 - accuracy: 0.0782 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 5/10
173/173 [==============================] - 1s 7ms/step - loss: 0.0000e+00 - accuracy: 0.0772 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 6/10
173/173 [==============================] - 1s 7ms/step - loss: 0.0000e+00 - accuracy: 0.0782 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 7/10
173/173 [==============================] - 1s 7ms/step - loss: 0.0000e+00 - accuracy: 0.0785 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 8/10
173/173 [==============================] - 1s 7ms/step - loss: 0.0000e+00 - accuracy: 0.0748 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 9/10
173/173 [==============================] - 1s 7ms/step - loss: 0.0000e+00 - accuracy: 0.0

87/87 [==============================] - 1s 9ms/step - loss: -933.6399 - accuracy: 0.0740 - val_loss: -1315.0023 - val_accuracy: 0.0789
Epoch 5/10
87/87 [==============================] - 1s 10ms/step - loss: -1519.9580 - accuracy: 0.0713 - val_loss: -1984.7617 - val_accuracy: 0.0789
Epoch 6/10
87/87 [==============================] - 1s 10ms/step - loss: -2237.3455 - accuracy: 0.0733 - val_loss: -2786.5471 - val_accuracy: 0.0789
Epoch 7/10
87/87 [==============================] - 1s 9ms/step - loss: -3124.7220 - accuracy: 0.0712 - val_loss: -3718.4106 - val_accuracy: 0.0789
Epoch 8/10
87/87 [==============================] - 1s 9ms/step - loss: -4039.7127 - accuracy: 0.0760 - val_loss: -4774.2041 - val_accuracy: 0.0789
Epoch 9/10
87/87 [==============================] - 1s 9ms/step - loss: -5212.4310 - accuracy: 0.0762 - val_loss: -5950.1294 - val_accuracy: 0.0789
Epoch 10/10
22/22 [==============================] - 0s 3ms/step - loss: -7380.8799 - accuracy: 0.0800
Epoch 1/10
690/690 

690/690 [==============================] - 3s 4ms/step - loss: 0.0000e+00 - accuracy: 0.0776 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 9/10
690/690 [==============================] - 3s 4ms/step - loss: 0.0000e+00 - accuracy: 0.0736 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 10/10
169/169 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0800
Epoch 1/10
345/345 [==============================] - 7s 7ms/step - loss: 0.0000e+00 - accuracy: 0.0758 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 2/10
345/345 [==============================] - 2s 5ms/step - loss: 0.0000e+00 - accuracy: 0.0711 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 3/10
345/345 [==============================] - 2s 6ms/step - loss: 0.0000e+00 - accuracy: 0.0754 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 4/10
345/345 [==============================] - 2s 5ms/step - loss: 0.0000e+00 - accuracy: 0.0787 - val_loss: 0.0000e+00 - val_accuracy: 0.0789
Epoch 

85/85 [==============================] - 0s 1ms/step - loss: -185.1372 - accuracy: 0.0800
Epoch 1/10
173/173 [==============================] - 6s 11ms/step - loss: -6.0231 - accuracy: 0.0758 - val_loss: -15.8012 - val_accuracy: 0.0789
Epoch 2/10
173/173 [==============================] - 1s 6ms/step - loss: -18.5214 - accuracy: 0.0758 - val_loss: -24.2144 - val_accuracy: 0.0789
Epoch 3/10
173/173 [==============================] - 1s 6ms/step - loss: -26.9690 - accuracy: 0.0773 - val_loss: -32.2593 - val_accuracy: 0.0789
Epoch 4/10
173/173 [==============================] - 1s 6ms/step - loss: -35.6249 - accuracy: 0.0769 - val_loss: -40.1792 - val_accuracy: 0.0789
Epoch 5/10
173/173 [==============================] - 1s 6ms/step - loss: -43.4069 - accuracy: 0.0743 - val_loss: -48.2887 - val_accuracy: 0.0789
Epoch 6/10
173/173 [==============================] - 1s 7ms/step - loss: -52.0606 - accuracy: 0.0707 - val_loss: -56.7196 - val_accuracy: 0.0789
Epoch 7/10
173/173 [==============

87/87 [==============================] - 1s 9ms/step - loss: 7.9056 - accuracy: 0.0761 - val_loss: 7.6648 - val_accuracy: 0.0789
Epoch 4/10
87/87 [==============================] - 1s 9ms/step - loss: 7.8530 - accuracy: 0.0771 - val_loss: 7.6648 - val_accuracy: 0.0789
Epoch 5/10
87/87 [==============================] - 1s 9ms/step - loss: 8.0109 - accuracy: 0.0784 - val_loss: 7.6648 - val_accuracy: 0.0789
Epoch 6/10
87/87 [==============================] - 1s 9ms/step - loss: 7.9514 - accuracy: 0.0778 - val_loss: 7.6648 - val_accuracy: 0.0789
Epoch 7/10
87/87 [==============================] - 1s 9ms/step - loss: 7.8691 - accuracy: 0.0748 - val_loss: 7.6648 - val_accuracy: 0.0789
Epoch 8/10
87/87 [==============================] - 1s 9ms/step - loss: 8.0651 - accuracy: 0.0732 - val_loss: 7.6648 - val_accuracy: 0.0789
Epoch 9/10
87/87 [==============================] - 1s 9ms/step - loss: 8.1531 - accuracy: 0.0804 - val_loss: 7.6648 - val_accuracy: 0.0789
Epoch 10/10
22/22 [============

In [76]:
maxacc_lstm = 0
for fa in first_activations:
    for sa in second_activations:
        for ta in third_activations:
            for los in losses:
                for batch in batch_sizes:
                    if accuracies[fa][sa][ta][los][batch][1] > maxacc_lstm:
                        maxacc_lstm = accuracies[fa][sa][ta][los][batch][1]
                        print(fa, sa, ta, los, batch)
print(maxacc_lstm)

gelu gelu gelu categorical_crossentropy 16
gelu gelu exponential categorical_crossentropy 16
0.14185185730457306


In [103]:
model = keras.Sequential()
model.add(keras.layers.LSTM(100, activation='gelu', return_sequences = True))
model.add(keras.layers.Dropout(.2))
model.add(keras.layers.LSTM(100, activation='gelu', return_sequences = True))
model.add(keras.layers.Dropout(.2))
model.add(keras.layers.LSTM(50, activation='exponential'))
model.add(keras.layers.Dropout(.2))
model.add(keras.layers.Dense(1, activation='softmax'))
model.compile(optimizer=keras.optimizers.Adam(learning_rate = .001, decay=.00001), metrics=['accuracy'], loss='categorical_crossentropy')
    
model.fit(x_train, y_train, batch_size=16, epochs=10, validation_data=(x_valid, y_valid))

model.evaluate(x_test, y_test, batch_size=16)

Epoch 1/10
690/690 [==============================] - 22s 11ms/step - loss: nan - accuracy: 0.1492 - val_loss: nan - val_accuracy: 0.1481
Epoch 2/10
690/690 [==============================] - 5s 8ms/step - loss: nan - accuracy: 0.1509 - val_loss: nan - val_accuracy: 0.1481
Epoch 3/10
690/690 [==============================] - 5s 7ms/step - loss: nan - accuracy: 0.1507 - val_loss: nan - val_accuracy: 0.1481
Epoch 4/10
690/690 [==============================] - 6s 8ms/step - loss: nan - accuracy: 0.1522 - val_loss: nan - val_accuracy: 0.1481
Epoch 5/10
690/690 [==============================] - 6s 8ms/step - loss: nan - accuracy: 0.1567 - val_loss: nan - val_accuracy: 0.1481
Epoch 6/10
690/690 [==============================] - 5s 8ms/step - loss: nan - accuracy: 0.1496 - val_loss: nan - val_accuracy: 0.1481
Epoch 7/10
690/690 [==============================] - 5s 8ms/step - loss: nan - accuracy: 0.1474 - val_loss: nan - val_accuracy: 0.1481
Epoch 8/10
690/690 [==========================

[nan, 0.14185185730457306]